# Convolutional Neural Networks

![sheme](https://res.mdpi.com/entropy/entropy-19-00242/article_deploy/html/images/entropy-19-00242-g001.png)

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense


In [0]:
# Создаем CNN
classifier = Sequential()

# Свертка (Convolution)
classifier.add(Convolution2D(32, (3, 3), strides=(1, 1), input_shape = (128, 128, 3), activation = 'relu'))

# Объединение (Pooling)
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Второй сверточный слой
classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Уплощение (Flattening)
classifier.add(Flatten())

# Полносвязянный слой (Full connection)
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Компиляция CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 45,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/data/dataset/training_set',
                                                 target_size = (128, 128),
                                                 batch_size = 40,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/data/dataset/test_set',
                                            target_size = (128, 128),
                                            batch_size = 40,
                                            class_mode = 'binary')



In [0]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 200,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 50)

In [0]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/gdrive/My Drive/Colab Notebooks/data/dataset/single_prediction/cat_or_dog_2.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [0]:
def dogorcat(result):
  if result[0][0] == 1:
        return 'dog'
  else:
        return 'cat'

In [0]:
import glob
from IPython.display import Image, display
for imageName in glob.glob('/content/gdrive/My Drive/Colab Notebooks/data/dataset/single_prediction/*'):
    display(Image(filename=imageName, width=350, ))
    test_image = image.load_img(imageName, target_size = (128, 128))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)    
    print(dogorcat(result))